In [ ]:
import sys,os
sys.path.append("..")
import django
django.setup()
import pandas as pd
from io import StringIO
import requests
import datetime
# from crawlers.models import *
# from crawlers.finlab.import_tools import *
from crawlers.finlab.data_process_tools import *

In [2]:
import pandas as pd
from io import StringIO
import requests
import datetime
import sys,os
class CrawlStockMarginTransactionsTW:
    def __init__(self, date):
        self.date = date
        self.date_str = date.strftime("%Y%m%d")
        self.target_name = "台股融資交易資訊"
        self.sub_market = ["sii", "otc"]
        
    def crawl_sii(self):
        r= requests.get('http://www.twse.com.tw/exchangeReport/MI_MARGN?response=csv&date=' +self.date_str+ '&selectType=ALL')
        content = r.text.replace('=', '')
        lines = content.split('\n')
        lines = list(filter(lambda l:len(l.split('",')) > 10, lines))
        content = "\n".join(lines)
        if content == '':
            return None
        df = pd.read_csv(StringIO(content)).dropna(how='all', axis=1).dropna(thresh=15)
        if len(df)<1:
            return None
        df = df.astype(str)
        df = df.apply(lambda s: s.str.replace(',', ''))
        df['股票代號'] = df['股票代號'].apply(lambda s:'00'+s if len(s)<3 else s)
        df = df.rename(columns={'股票代號':'stock_id','股票名稱':'stock_name',
                                '買進':'mt_buy','賣出':'mt_sell',
                                '現金償還':'cash_redemption','前日餘額':'mt_balance_pd',
                                '今日餘額':'mt_balance_now','限額':'mt_quota',
                                '買進.1':'short_covering','賣出.1':'short_sale',
                                '現券償還':'stock_redemption','前日餘額.1':'ss_balance_pd',
                                '今日餘額.1':'ss_balance_now','限額.1':'ss_quota',
                                '資券互抵':'offset','註記':'note'})
        df.iloc[:,2:-1]=df.iloc[:,2:-1].apply(lambda s:pd.to_numeric(s,errors='coerce'))

#         df['note']=[symbols_change(i,{'O':'停止融資','X':'停止融券','@':'融資分配','%':'融券分配','!':'停止買賣'}) for i in df['note']]
        df['mt_use_rate']=[round(v/q*100,2) if q>0 else 100 if v>0 else 0 for v,q in zip (df['mt_balance_now'],df['mt_quota'])]
        df['ss_use_rate']=[round(v/q*100,2) if q>0 else 100 if v>0 else 0 for v,q in zip (df['ss_balance_now'],df['ss_quota'])]
        df['date'] = pd.to_datetime(self.date).date()
        return df
    
    def crawl_otc(self):
        y = str(int(self.date.strftime("%Y")) - 1911)
        date_str = y + "/" + self.date.strftime("%m") + "/" + self.date.strftime("%d")
        r = requests.get('http://www.tpex.org.tw/web/stock/margin_trading/margin_balance/margin_bal_result.php?l=zh-tw&o=csv&d='+date_str+'&s=0,asc,0')
        df = pd.read_csv(StringIO(r.text), header=2).dropna(how='all', axis=1).dropna(thresh=15)
        if len(df)<10:
            return None
        df = df.astype(str).apply(lambda s: s.str.replace(',',''))
        df.columns=[col.replace(' ','') for col in df.columns]
        df['代號'] = df['代號'].apply(lambda s:'00'+s if len(s)<3 else s)
        df = df.rename(columns={'代號':'stock_id','名稱':'stock_name',
                                '資買':'mt_buy','資賣':'mt_sell',
                                '現償':'cash_redemption','前資餘額(張)':'mt_balance_pd',
                                '資餘額':'mt_balance_now','資限額':'mt_quota',
                                '券買':'short_covering','券賣':'short_sale',
                                '券償':'stock_redemption','前券餘額(張)':'ss_balance_pd',
                                '券餘額':'ss_balance_now','券限額':'ss_quota',
                                '資券相抵(張)':'offset','備註':'note',
                                '資使用率(%)':'mt_use_rate','券使用率(%)':'ss_use_rate'})
        df=df.drop(columns=['資屬證金','券屬證金'])
        df.iloc[:,2:-1]=df.iloc[:,2:-1].apply(lambda s:pd.to_numeric(s,errors='coerce'))
#         df['note']=[symbols_change(i,{'O':'停止融資','X':'停止融券','@':'融資分配','%':'融券分配','!':'停止買賣',
#                                     '*':'融券餘額占融資餘額百分之六十以上者','A':'股價波動過度劇烈','B':'股權過度集中',
#                                       'C':'成交量過度異常','D':'監視第二次處置'}) for i in df['note']]
        df['date'] = pd.to_datetime(self.date).date()
        return df

    def crawl_main(self):
        try:
            df = pd.concat([self.crawl_sii(),self.crawl_otc()],sort=False)
        except ValueError:
            return None
        return df
    

import datetime
df=CrawlStockMarginTransactionsTW(datetime.datetime(2020,6,9)).crawl_main()
df

,stock_id,stock_name,mt_buy,mt_sell,cash_redemption,mt_balance_pd,mt_balance_now,mt_quota,short_covering,short_sale,stock_redemption,ss_balance_pd,ss_balance_now,ss_quota,offset,note,mt_use_rate,ss_use_rate,date
0,0050,元大台灣50,97,531,0,1689,1255,284625,24,1,0,900,877,284625,4,,0.44,0.31,2020-06-09
1,0051,元大中型100,0,0,0,28,28,2500,0,0,0,0,0,2500,0,,1.12,0,2020-06-09
2,0052,富邦科技,0,12,0,54,42,6875,0,0,0,0,0,6875,0,,0.61,0,2020-06-09
3,0053,元大電子,0,0,0,0,0,1622,0,0,0,0,0,1622,0,X,0,0,2020-06-09
4,0054,元大台商50,0,0,0,0,0,2531,0,0,0,0,0,2531,0,X,0,0,2020-06-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
741,9949,琉園,31,0,0,6395,6426,11758,0,0,0,0,0,11758,0,nan,54.65,0,2020-06-09
742,9950,萬國通,0,0,55,55,0,0,0,0,0,0,0,0,0,OX,0,0,2020-06-09
743,9951,皇田,170,157,0,1068,1081,18725,0,0,0,2,2,18725,0,nan,5.77,0.01,2020-06-09
744,9960,邁達康,0,1,0,4787,4786,8398,0,0,0,0,0,8398,0,11 BC,56.98,0,2020-06-09


In [ ]:
df=CrawlStockTiiMarketReportTW(datetime.datetime(2020,4,29)).crawl_main()
add_to_sql(StockTiiMarketReportTW,df)